# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [82]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [83]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [84]:
from iex_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [85]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 80620195, 'calculationPrice': 'close', 'change': -0.37, 'changePercent': -0.00266, 'close': 147.78, 'closeSource': 'lcioaiff', 'closeTime': 1706030276393, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 151.734, 'delayedPriceTime': 1736260187221, 'extendedChange': -1.18, 'extendedChangePercent': -0.00803, 'extendedPrice': 145.93, 'extendedPriceTime': 1735966158068, 'high': 148.93, 'highSource': 'ndr5u  ey1im ceaedptile', 'highTime': 1674636473956, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 146.03, 'iexCloseTime': 1720474682404, 'iexLastUpdated': 1668690712620, 'iexMarketPercent': 0.019650850567259168, 'iexOpen': 148.15, 'iexOpenTime': 1720069376572, 'iexRealtimePrice': 152.62, 'iexRealtimeSize': 101, 'iexVolume': 1354558, 'lastTradeTime': 1719991741705, 'latestPrice': 145.77, 'latestSource': 'Close', 'latestTime': 'July 13, 2022', 'latestUpdate': 1698461064511, 'latestVolume': 73270718, 'low': 147.13, 'lowSource':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [86]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/100000000)

24138.36577508


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [87]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [88]:
final_dataframe.append(
    pd.Series([symbol,price,market_cap,'N/A'], index = my_columns),
    ignore_index=True,
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,145.77,2413836577508,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [89]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol in stocks['Ticker'][0:10]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([symbol,data['latestPrice'],data['marketCap'],'N/A'], index = my_columns),
        ignore_index=True,
    )

In [90]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.42,35488386232,N/A
1,AAL,14.95,9447043787,N/A
2,AAP,185.92,11545532145,N/A
3,AAPL,147.76,2457039363967,N/A
4,ABBV,156.51,277570079146,N/A
5,ABC,140.73,29599899509,N/A
6,ABMD,259.41,12308036008,N/A
7,ABT,106.91,190255833856,N/A
8,ACN,275.35,186003617608,N/A
9,ADBE,382.37,177126078591,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [91]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [92]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)
for symbols in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?token={IEX_CLOUD_API_TOKEN}&symbols={symbols}&types=quote'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    # print(data)
    for symbol in data:
        # print(symbol)
        # print(data[symbol]['quote']['latestPrice'])
        # print(data[symbol]['quote']['marketCap'])
        final_dataframe = final_dataframe.append(
            pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'], index = my_columns),
            ignore_index=True,
        )
final_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch/?token=Tpk_0020df4fd968417db3419f6708aba4fa&symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote
https://sandbox.iexapis.com/stable/stock/market/batch/?token=Tpk_0020df4fd968417db3419f6708aba4fa&symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FI

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.88,36680910716,N/A
1,AAL,14.70,9618411347,N/A
2,AAP,193.96,11459445581,N/A
3,AAPL,148.39,2418666273229,N/A
4,ABBV,153.51,271877173378,N/A
...,...,...,...,...
496,YUM,121.25,33982580339,N/A
497,ZBH,102.56,22504110838,N/A
498,ZBRA,306.01,15807514568,N/A
499,ZION,51.01,7625827995,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [93]:
portfolio_size = input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number \nPlease try again")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)


In [94]:
position_size = val/len(final_dataframe.index)
# position_size
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,121.88,36680910716,163
1,AAL,14.70,9618411347,1357
2,AAP,193.96,11459445581,102
3,AAPL,148.39,2418666273229,134
4,ABBV,153.51,271877173378,130
...,...,...,...,...
496,YUM,121.25,33982580339,164
497,ZBH,102.56,22504110838,194
498,ZBRA,306.01,15807514568,65
499,ZION,51.01,7625827995,391


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [112]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [113]:
bg_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
    'font_color': font_color,
    'bg_color': bg_color,
    'border': 1
})
dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': bg_color,
    'border': 1
})
integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': bg_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [110]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', string_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [114]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}
for col in column_formats:
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 18, column_formats[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1', column_formats[col][0], column_formats[col][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()